# Capstone Week 3 Peer-Graded Assignment

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

import matplotlib as plt 
%matplotlib inline

print('All libraries succesfully imported')

All libraries succesfully imported


In [2]:
# install goecoder
!pip install geocoder
import geocoder

### scrape neighbourhood data from wiki and import into df

In [3]:
# get Toronto neighbourhoods data
from pandas.io.html import read_html
Toronto_Neighbourhoods = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
neigh = pd.read_html(Toronto_Neighbourhoods, attrs = {'class':'wikitable'})
df = neigh[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
postal_data = pd.read_csv('https://cocl.us/Geospatial_data')
postal_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### add lat/long to df 

In [72]:
#join df and postal_data dataframes by Postal Code
nb = df.set_index('Postal Code').join(postal_data.set_index('Postal Code'))
nb.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M1A,Not assigned,Not assigned,NaN,NaN
M2A,Not assigned,Not assigned,NaN,NaN
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636


### finish cleaning data

In [73]:
#drop rows where Borough == 'Not assigned'
nb.drop(nb.loc[nb['Borough'] == 'Not assigned'].index, inplace = True)
nb.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [74]:
#replace 'Not assigned' values for Neighborhood with Borough value
nb['Neighborhood'].replace('Not assigned', nb['Borough'], inplace = True)
nb.head()

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [75]:
#get matrix size and check duplicate neighborhoods which will be a pain to deal with later
nb_unique = len(nb['Neighborhood'].unique())
nb_total = len(nb['Neighborhood'])
print('{} unique neighborhood vs {} neighborhood'.format(nb_unique, nb_total))
print(nb[nb.duplicated(subset= 'Neighborhood')])

99 unique neighborhood vs 103 neighborhood
                Borough Neighborhood   Latitude  Longitude
Postal Code                                               
M3C          North York    Don Mills  43.725900 -79.340923
M3L          North York    Downsview  43.739015 -79.506944
M3M          North York    Downsview  43.728496 -79.495697
M3N          North York    Downsview  43.761631 -79.520999


In [76]:
#drop duplicate neighborhoods
# nb.drop_duplicates(subset = 'Neighborhood', inplace = True)
# nb.shape

# Clustering and data viz

In [9]:
import requests
from pandas.io.json import json_normalize #transform json into df 

#import viz libraries
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes
import folium #map rendering 

#some other stuff for geo
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::openssl-1.1.1d-he774522_2done

# All requested packages already installed.



# Create map showing all Toronto neighborhoods

In [62]:
# create map
address = 'Toronto, ON'
geolocator = Nominatim(user_agent = 'toronto_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

# add markers
for lat,lng, borough, neighborhood in zip(nb['Latitude'], nb['Longitude'], nb['Borough'], nb['Neighborhood']): 
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

In [11]:
# Foursquare creds
CLIENT_ID = 'DCDD3QL33RRCZ10MIOOMYSRIQYMIVOJOLS0J0XS2EVKKIKWO'
CLIENT_SECRET = '3ZVG0WN4TMYCOYRNPMXOJES3SJVAS4KHAWIHBKGWVHR1YTES'
VERSION = '20200602' #API version 

# Create function to get nearby venues for all neighborhoods

In [77]:
LIMIT = 100 
radius = 500 

def getNearbyVenues(names, latitude, longitude, radius=500): 
    venues_list=[]
    for name, lat, lng in zip(names, latitude, longitude): 
        print(name)
        #API request url
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        #GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
        name, 
        lat, 
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude', 
                             'Neighborhood Longitude', 
                             'Venue',
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category']
                    
    
    return(nearby_venues)

In [78]:
toronto_venues = getNearbyVenues(names=nb['Neighborhood'], 
                                latitude = nb['Latitude'], 
                                longitude = nb['Longitude']
                                )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [79]:
len(toronto_venues['Neighborhood'].unique())

96

In [80]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,5,5,5,5,5,5
Woodbine Heights,7,7,7,7,7,7


In [81]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


# Create dummies for each venue type

In [82]:
# get categories dummies
one_hot_toronto = pd.get_dummies(toronto_venues[['Venue Category']],prefix ="", prefix_sep="" )
one_hot_toronto['Neighborhood'] = toronto_venues["Neighborhood"]

fixed_columns = [one_hot_toronto.columns[-1]] + list(one_hot_toronto.columns[:-1])
one_hot_toronto = one_hot_toronto[fixed_columns]

one_hot_toronto.shape

(2129, 272)

In [83]:
toronto_grouped = one_hot_toronto.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
# top 5 mot common venues by neighborhood 
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']: 
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2  Latin American Restaurant   0.2
3             Clothing Store   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1          Pool  0.11
2           Pub  0.11
3   Coffee Shop  0.11
4  Skating Rink  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0              Bank  0.10
1       Coffee Shop  0.10
2             Diner  0.05
3  Sushi Restaurant  0.05
4       Supermarket  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1      Sandwich Place  0.09
2          Restaurant  0.09
3         Coffee Shop  0.09
4   India

                    venue  freq
0             Coffee Shop  0.09
1            Burger Joint  0.06
2                    Bank  0.06
3     Sporting Goods Shop  0.06
4  Furniture / Home Store  0.06


----Little Portugal, Trinity----
                           venue  freq
0                            Bar  0.11
1               Asian Restaurant  0.07
2  Vegetarian / Vegan Restaurant  0.05
3                     Restaurant  0.05
4                    Coffee Shop  0.05


----Malvern, Rouge----
                             venue  freq
0             Fast Food Restaurant   1.0
1                    Metro Station   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                venue  freq
0          Playground  0.33
1                Park  0.33
2         Coffee Shop  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Mimico NW, The Queensway We

4       Pharmacy   0.2


----Woburn----
               venue  freq
0        Coffee Shop   0.4
1           Pharmacy   0.2
2       Soccer Field   0.2
3  Korean Restaurant   0.2
4     Massage Studio   0.0


----Woodbine Heights----
                venue  freq
0               Diner  0.14
1         Curling Ice  0.14
2          Beer Store  0.14
3        Skating Rink  0.14
4  Athletics & Sports  0.14


----York Mills West----
                             venue  freq
0                             Park   0.5
1                Convenience Store   0.5
2                      Yoga Studio   0.0
3                    Metro Station   0.0
4  Molecular Gastronomy Restaurant   0.0


----York Mills, Silver Hills----
                             venue  freq
0                        Cafeteria   1.0
1               Mexican Restaurant   0.0
2              Monument / Landmark   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0




# Get most common venues by neighborhood

In [85]:
# putting most common venues into df

def return_most_common_venues(row, num_top_venues): 
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [93]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues): 
    try: 
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except: 
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# new df
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]): 
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Skating Rink,Clothing Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Skating Rink,Pharmacy,Pub,Pool,Women's Store,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Middle Eastern Restaurant,Pizza Place,Ice Cream Shop,Sushi Restaurant,Restaurant,Deli / Bodega,Mobile Phone Shop,Fried Chicken Joint
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Department Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Pizza Place,Juice Bar,Butcher,Indian Restaurant,Café,Sushi Restaurant


# Time for some k-means action. Good diversity of neighborhoods in Toronto so let's break them into 10 clusters


In [94]:
kclusters = 10

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# do some k-means 
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# check labels
toronto_grouped_clustering.shape

(96, 271)

# Join df to include Cluster Labels 

In [95]:
# add cluster labels
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)

toronto_merged = nb
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.dropna(inplace = True)
toronto_merged['Cluster Labels'] = [int(x) for x in toronto_merged['Cluster Labels']]
toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Postal Code,,,,,,,,,,,,,,,
M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
M4A,North York,Victoria Village,43.725882,-79.315572,9,French Restaurant,Coffee Shop,Portuguese Restaurant,Hockey Arena,Distribution Center,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,9,Coffee Shop,Pub,Bakery,Park,Theater,Restaurant,Breakfast Spot,Café,Event Space,Performing Arts Venue
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,9,Accessories Store,Women's Store,Coffee Shop,Event Space,Miscellaneous Shop,Athletics & Sports,Arts & Crafts Store,Clothing Store,Furniture / Home Store,Boutique
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,9,Coffee Shop,Sushi Restaurant,Yoga Studio,College Cafeteria,Bar,Beer Bar,Smoothie Shop,Burger Joint,Sandwich Place,Burrito Place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,1,River,Pool,Women's Store,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,9,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Hotel,Bubble Tea Shop,Burger Joint
M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,1,Light Rail Station,Gym / Fitness Center,Smoke Shop,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden


In [96]:
toronto_merged['Cluster Labels'].unique()

array([0, 9, 4, 1, 5, 6, 2, 8, 7, 3], dtype=int64)

# Viz this bad boy

In [98]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

(103, 4)